## SkipGram -> feed in the center word and predict the context words


**The X value is Single Centre Word and Y value is the set of Context words**

**Importing libraries**

In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.layers import add, dot, concatenate, Permute
from keras.callbacks import ModelCheckpoint

from numpy import argmax
from keras.utils import to_categorical


Using TensorFlow backend.


In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/swati/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/swati/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
class FirstWordException(Exception):
    '''raised when first word is encountered'''
    pass 

**getting data**

In [4]:
with open('/home/swati/Word2Vec/data/text8', 'r') as f:
    wikipedia=f.read()

stopwords=set(stopwords.words('english')+['as', 'a', 'b', 'of', ''])

In [5]:
wiki = nltk.word_tokenize(wikipedia)

**cleaning data**

In [28]:
# wiki=[x.lower() for x in wiki if x not in stopwords and len(x)>2]
wiki=[x.lower() for x in wiki if x not in stopwords and len(x)>=2]
words=wiki[:1000]

# words=[x for x in words if len(x)>2]

vocab_len=len(set(words))
vocab_len

561

**creating dictionary for vocabulary**

In [17]:
vocabulary=set(words)
word_index=dict((c, i + 1) for i, c in enumerate(vocabulary))

In [24]:
def oneHotEncode(word):
    word_vec=[0 for i in range(len(vocabulary))]
    word_ind = word_index[word]-1
    word_vec[word_ind] = 1
    return word_vec

**creating pairs of neighbouring words**

In [133]:
def create_pairs(words, wikipedia, window_size=5):
    keyword=''
    training_data=[]
    for word in words:
        word_ind =wikipedia.index(word)
        forward_ind=int(window_size/2)
        w_target=[]
        w_context=[]
        for i in range(1,forward_ind+1):
            try:
                w_context.append(oneHotEncode(wikipedia[word_ind+i]))
                w_target.append(oneHotEncode(word))
#                 print(len(oneHotEncode(wikipedia[word_ind+i])))
#                 print(len(oneHotEncode(word)))
#                 training_data.append([oneHotEncode(wikipedia[word_ind+i]), oneHotEncode(word)])
            except:
                pass
            try: 
                if (word_ind-i<0):
                    raise FirstWordException
                w_context.append(oneHotEncode(wikipedia[word_ind-i]))
                w_target.append(oneHotEncode(word))
#                 print(len(oneHotEncode(wikipedia[word_ind-i])))
#                 print(len(oneHotEncode(word)))
#                 training_data.append([oneHotEncode(wikipedia[word_ind-i]), oneHotEncode(word)])
            except:
                pass
            finally:
                w_target=oneHotEncode(word)
#                 training_data.append([w_target, w_context])
        training_data.append([w_target, w_context])
      
    return np.array(training_data)

In [134]:
training_data=create_pairs(words, words, 5)

In [135]:
training_data.shape

(1000, 2)

In [137]:
max_c_words=max([len(training_data[i][1]) for i in range(training_data.shape[0])])

In [138]:
max_c_words

4

**Padding values to maximum val length**

In [139]:
word_vec=[0 for i in range(len(vocabulary))]
for w_t, w_c in training_data:
    if(len(w_c)<max_c_words):
        for i in range(max_c_words-len(w_c)):
            w_c.append(word_vec)

In [140]:
len(training_data[0][1])

4

**Splitting Training and Test Data using CBOW method**

In [141]:
X=[]
y=[]
for w_t, w_c in training_data:
    X.append(w_t)
    y.append(w_c)

In [147]:
X=np.array(X)
y=np.array(y).reshape(1000,-1)
X.shape, y.shape

((1000, 561), (1000, 2244))

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

**Reshaping 3D array to 2D array**

In [150]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((670, 561), (330, 561), (670, 2244), (330, 2244))

In [166]:
model = Sequential()
model.add(Dense(200, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(130, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

In [167]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

In [173]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')



hist = model.fit(X_train, y_train, epochs = 500, batch_size = 64
                 , validation_data = (X_test, y_test), callbacks = [checkpoint])

Train on 670 samples, validate on 330 samples
Epoch 1/500
670/670 [==============================] - 0s 191us/step - loss: 0.0018 - acc: 0.0045 - val_loss: 0.0018 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 0.00176, saving model to model.h5
Epoch 2/500
670/670 [==============================] - 0s 159us/step - loss: 0.0018 - acc: 0.0075 - val_loss: 0.0018 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 0.00176 to 0.00176, saving model to model.h5
Epoch 3/500
670/670 [==============================] - 0s 157us/step - loss: 0.0018 - acc: 0.0045 - val_loss: 0.0018 - val_acc: 0.0152

Epoch 00003: val_loss improved from 0.00176 to 0.00176, saving model to model.h5
Epoch 4/500
670/670 [==============================] - 0s 159us/step - loss: 0.0018 - acc: 0.0075 - val_loss: 0.0018 - val_acc: 0.0152

Epoch 00004: val_loss improved from 0.00176 to 0.00176, saving model to model.h5
Epoch 5/500
670/670 [==============================] - 0s 159us/step - loss: 0.001

670/670 [==============================] - 0s 174us/step - loss: 0.0017 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00040: val_loss did not improve from 0.00175
Epoch 41/500
670/670 [==============================] - 0s 155us/step - loss: 0.0017 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00041: val_loss did not improve from 0.00175
Epoch 42/500
670/670 [==============================] - 0s 173us/step - loss: 0.0017 - acc: 0.0060 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00042: val_loss did not improve from 0.00175
Epoch 43/500
670/670 [==============================] - 0s 167us/step - loss: 0.0017 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00043: val_loss improved from 0.00175 to 0.00175, saving model to model.h5
Epoch 44/500
670/670 [==============================] - 0s 166us/step - loss: 0.0017 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00044: val_loss improved from 0.00175 to 0.00174, saving model to mo

670/670 [==============================] - 0s 152us/step - loss: 0.0017 - acc: 0.0149 - val_loss: 0.0017 - val_acc: 0.0273

Epoch 00078: val_loss improved from 0.00170 to 0.00170, saving model to model.h5
Epoch 79/500
670/670 [==============================] - 0s 156us/step - loss: 0.0017 - acc: 0.0104 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00079: val_loss did not improve from 0.00170
Epoch 80/500
670/670 [==============================] - 0s 160us/step - loss: 0.0017 - acc: 0.0224 - val_loss: 0.0017 - val_acc: 0.0182

Epoch 00080: val_loss did not improve from 0.00170
Epoch 81/500
670/670 [==============================] - 0s 154us/step - loss: 0.0017 - acc: 0.0358 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00081: val_loss did not improve from 0.00170
Epoch 82/500
670/670 [==============================] - 0s 163us/step - loss: 0.0016 - acc: 0.0224 - val_loss: 0.0017 - val_acc: 0.0030

Epoch 00082: val_loss did not improve from 0.00170
Epoch 83/500
670/670 [=====================


Epoch 00119: val_loss did not improve from 0.00169
Epoch 120/500
670/670 [==============================] - 0s 158us/step - loss: 0.0016 - acc: 0.0284 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00120: val_loss did not improve from 0.00169
Epoch 121/500
670/670 [==============================] - 0s 159us/step - loss: 0.0016 - acc: 0.0179 - val_loss: 0.0017 - val_acc: 0.0030

Epoch 00121: val_loss improved from 0.00169 to 0.00169, saving model to model.h5
Epoch 122/500
670/670 [==============================] - 0s 163us/step - loss: 0.0016 - acc: 0.0119 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00122: val_loss improved from 0.00169 to 0.00169, saving model to model.h5
Epoch 123/500
670/670 [==============================] - 0s 157us/step - loss: 0.0016 - acc: 0.0239 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00123: val_loss improved from 0.00169 to 0.00169, saving model to model.h5
Epoch 124/500
670/670 [==============================] - 0s 154us/step - loss: 0.0016 - acc: 0

670/670 [==============================] - 0s 157us/step - loss: 0.0016 - acc: 0.0000e+00 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00159: val_loss improved from 0.00168 to 0.00168, saving model to model.h5
Epoch 160/500
670/670 [==============================] - 0s 188us/step - loss: 0.0016 - acc: 0.0060 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00160: val_loss did not improve from 0.00168
Epoch 161/500
670/670 [==============================] - 0s 175us/step - loss: 0.0016 - acc: 0.0030 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00161: val_loss improved from 0.00168 to 0.00168, saving model to model.h5
Epoch 162/500
670/670 [==============================] - 0s 156us/step - loss: 0.0016 - acc: 0.0000e+00 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00162: val_loss improved from 0.00168 to 0.00168, saving model to model.h5
Epoch 163/500
670/670 [==============================] - 0s 155us/step - loss: 0.0016 - acc: 0.0000e+00 - val_loss: 0.0017 - val_acc: 0.0000e+00

...

670/670 [==============================] - 0s 152us/step - loss: 0.0016 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00321: val_loss did not improve from 0.00167
Epoch 322/500
670/670 [==============================] - 0s 204us/step - loss: 0.0016 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00322: val_loss did not improve from 0.00167
Epoch 323/500
670/670 [==============================] - 0s 181us/step - loss: 0.0016 - acc: 0.0060 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00323: val_loss did not improve from 0.00167
Epoch 324/500
670/670 [==============================] - 0s 199us/step - loss: 0.0016 - acc: 0.0090 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00324: val_loss did not improve from 0.00167
Epoch 325/500
670/670 [==============================] - 0s 148us/step - loss: 0.0016 - acc: 0.0030 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00325: val_loss did not improve from 0.00167
Epoch 326/500
670/670 [==============================] - 0s 1

670/670 [==============================] - 0s 150us/step - loss: 0.0016 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00363: val_loss did not improve from 0.00167
Epoch 364/500
670/670 [==============================] - 0s 140us/step - loss: 0.0016 - acc: 0.0030 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00364: val_loss did not improve from 0.00167
Epoch 365/500
670/670 [==============================] - 0s 142us/step - loss: 0.0016 - acc: 0.0164 - val_loss: 0.0017 - val_acc: 0.0152

Epoch 00365: val_loss did not improve from 0.00167
Epoch 366/500
670/670 [==============================] - 0s 141us/step - loss: 0.0016 - acc: 0.0254 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00366: val_loss did not improve from 0.00167
Epoch 367/500
670/670 [==============================] - 0s 143us/step - loss: 0.0016 - acc: 0.0000e+00 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00367: val_loss did not improve from 0.00167
Epoch 368/500
670/670 [==========================

670/670 [==============================] - 0s 145us/step - loss: 0.0016 - acc: 0.0060 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00405: val_loss did not improve from 0.00167
Epoch 406/500
670/670 [==============================] - 0s 149us/step - loss: 0.0016 - acc: 0.0030 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00406: val_loss did not improve from 0.00167
Epoch 407/500
670/670 [==============================] - 0s 146us/step - loss: 0.0016 - acc: 0.0075 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00407: val_loss did not improve from 0.00167
Epoch 408/500
670/670 [==============================] - 0s 143us/step - loss: 0.0016 - acc: 0.0045 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00408: val_loss did not improve from 0.00167
Epoch 409/500
670/670 [==============================] - 0s 142us/step - loss: 0.0016 - acc: 0.0045 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00409: val_loss did not improve from 0.00167
Epoch 410/500
670/670 [==========================

670/670 [==============================] - 0s 138us/step - loss: 0.0016 - acc: 0.0030 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00447: val_loss did not improve from 0.00167
Epoch 448/500
670/670 [==============================] - 0s 143us/step - loss: 0.0016 - acc: 0.0075 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00448: val_loss did not improve from 0.00167
Epoch 449/500
670/670 [==============================] - 0s 140us/step - loss: 0.0016 - acc: 0.0075 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00449: val_loss did not improve from 0.00167
Epoch 450/500
670/670 [==============================] - 0s 144us/step - loss: 0.0016 - acc: 0.0015 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00450: val_loss did not improve from 0.00167
Epoch 451/500
670/670 [==============================] - 0s 145us/step - loss: 0.0016 - acc: 0.0060 - val_loss: 0.0017 - val_acc: 0.0212

Epoch 00451: val_loss did not improve from 0.00167
Epoch 452/500
670/670 [==============================

670/670 [==============================] - 0s 150us/step - loss: 0.0016 - acc: 0.0075 - val_loss: 0.0017 - val_acc: 0.0212

Epoch 00489: val_loss did not improve from 0.00167
Epoch 490/500
670/670 [==============================] - 0s 138us/step - loss: 0.0016 - acc: 0.0119 - val_loss: 0.0017 - val_acc: 0.0212

Epoch 00490: val_loss did not improve from 0.00167
Epoch 491/500
670/670 [==============================] - 0s 141us/step - loss: 0.0016 - acc: 0.0090 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00491: val_loss did not improve from 0.00167
Epoch 492/500
670/670 [==============================] - 0s 144us/step - loss: 0.0016 - acc: 0.0045 - val_loss: 0.0017 - val_acc: 0.0000e+00

Epoch 00492: val_loss did not improve from 0.00167
Epoch 493/500
670/670 [==============================] - 0s 144us/step - loss: 0.0016 - acc: 0.0045 - val_loss: 0.0017 - val_acc: 0.0212

Epoch 00493: val_loss did not improve from 0.00167
Epoch 494/500
670/670 [==============================] - 0s 1